In [7]:
!pip install onnxruntime tf2onnx

You should consider upgrading via the 'c:\users\hadeel gamaleldin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
import os

# Define the MLP model path (The one used for landmarks)
model_path = "asl_mediapipe_mlp_model.h5" 
output_path = "mlp_model.onnx" # Naming it differently to avoid confusion

if os.path.exists(model_path):
    print(f"🔄 Converting {model_path} to ONNX...")
    # Convert the Keras model to ONNX
    os.system(f"python -m tf2onnx.convert --keras {model_path} --output {output_path}")
    print("✅ Success: 'mlp_model.onnx' created.")
else:
    print(f"❌ Error: Could not find {model_path}. Make sure the file is in the folder.")

🔄 Converting asl_mediapipe_mlp_model.h5 to ONNX...
✅ Success: 'mlp_model.onnx' created.


The Main Application (Your Interface)

In [6]:

# Cell 3: The Main Application
import cv2
import multiprocessing as mp
import time
# Import the class from the file we just created
from performance_engine import AIWorker 

def run_realtime_app():
    # 1. Setup Communication Queues (Size 1 to prevent lag buildup)
    frame_queue = mp.Queue(maxsize=1)
    result_queue = mp.Queue(maxsize=1)

    # 2. Start the Background AI Worker
    # We pass "mlp_model.onnx" which matches the Landmark data logic
    worker = AIWorker(frame_queue, result_queue, "mlp_model.onnx")
    worker.start()

    # 3. Start Camera (DirectShow for Windows speed)
    #cap = cv2.VideoCapture('http://192.168.100.33:4747/video')
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    # Check if the connection was actually made
    if not cap.isOpened():
        print("❌ ERROR: Python cannot find a camera at Index 1.")
        print("   -> Try changing index to 0.")
        print("   -> Try the IP Address method below.")
        worker.terminate() # Clean up the AI process
        return

    # Check if the camera is sending frames (The "Silent Crash" detector)
    ret, test_frame = cap.read()
    if not ret:
        print("❌ ERROR: Camera is connected, but sending BLANK frames.")
        print("   -> DroidCam Client might be active but 'Stopped'. Click 'Start' in the PC app.")
        print("   -> USB cable might be loose.")
        worker.terminate()
        return

    print("✅ Camera connection successful! Starting main loop...")
    
    print("🚀 App Started. Camera running at Max FPS. AI in background.")
    
    # UI State
    current_text = ""
    current_conf = 0.0
    warning = False
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret: break
            
            # Mirror effect
            frame = cv2.flip(frame, 1)

            # 4. SEND TO AI (Non-blocking fire-and-forget)
            if frame_queue.empty():
                frame_queue.put(frame)

            # 5. RETRIEVE FROM AI (Non-blocking check)
            if not result_queue.empty():
                current_text, current_conf, warning = result_queue.get()

            # 6. DRAWING (Runs at full 60 FPS independent of AI speed)
            h, w, _ = frame.shape
            
            # Black UI Bar
            cv2.rectangle(frame, (0, h-60), (w, h), (0,0,0), -1)
            
            if warning:
                 cv2.putText(frame, "Only 1 Hand Allowed!", (20, h-20), 
                             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            else:
                 cv2.putText(frame, current_text, (20, h-20), 
                             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            # Confidence Monitor (Top Left)
            color = (0, 255, 0) if current_conf > 0.7 else (0, 165, 255)
            cv2.putText(frame, f"Conf: {current_conf:.2f}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            cv2.imshow("High Performance ASL", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        pass
    finally:
        cap.release()
        cv2.destroyAllWindows()
        worker.terminate() # Kill background process
        print("✅ Clean Exit")

if __name__ == "__main__":
    run_realtime_app()

✅ Camera connection successful! Starting main loop...
🚀 App Started. Camera running at Max FPS. AI in background.
✅ Clean Exit


UNIVERSAL SELF HEALING CONNECTOR

In [3]:
import cv2
import multiprocessing as mp
import threading
import time
import queue
from performance_engine import AIWorker 

# --- CLASS: FAST THREADED CAMERA ---
class FrameReader(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.frame_queue = queue.Queue(maxsize=1) # Only keep latest frame
        self.running = True
        self.cap = None
        self.connected_source = None

    def run(self):
        # 1. ROBUST CONNECTION LOGIC (Try sources in order)
        # Replace '192.168.1.5' with your actual DroidCam IP if you want WiFi
        potential_sources = [
            ('http://192.168.1.5:4747/video', "WiFi IP"), 
            (1, "USB Index 1"), 
            (0, "USB Index 0")
        ]
        
        for source, name in potential_sources:
            print(f"📷 Trying connection to: {name}...")
            # Use CAP_DSHOW for USB on Windows for speed, normal for IP
            if isinstance(source, int):
                temp_cap = cv2.VideoCapture(source, cv2.CAP_DSHOW)
            else:
                temp_cap = cv2.VideoCapture(source)
                
            if temp_cap.isOpened():
                ret, _ = temp_cap.read()
                if ret:
                    print(f"✅ SUCCESS: Connected to {name}")
                    self.cap = temp_cap
                    self.connected_source = name
                    break
                else:
                    temp_cap.release()
            
        if self.cap is None:
            print("❌ FATAL: No Camera Found. Check DroidCam.")
            self.running = False
            return

        # 2. OPTIMIZE CAMERA SETTINGS (Crucial for Speed)
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1) # Don't buffer old frames
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        # 3. FAST CAPTURE LOOP
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                continue
            
            # Put new frame in queue, remove old one if full (Always fresh)
            if not self.frame_queue.empty():
                try: self.frame_queue.get_nowait()
                except: pass
            
            self.frame_queue.put(frame)

    def get_frame(self):
        try:
            return self.frame_queue.get_nowait()
        except:
            return None

    def stop(self):
        self.running = False
        if self.cap: self.cap.release()

# --- MAIN APP ---
def run_realtime_app():
    # 1. Start AI Engine
    frame_queue = mp.Queue(maxsize=1)
    result_queue = mp.Queue(maxsize=1)
    worker = AIWorker(frame_queue, result_queue, "mlp_model.onnx")
    worker.start()

    # 2. Start Fast Camera Thread
    print("🚀 Starting Fast Camera Thread...")
    cam_thread = FrameReader()
    cam_thread.start()
    
    # Wait a moment for camera to warm up
    time.sleep(2)
    
    if not cam_thread.running:
        print("❌ Camera failed to start.")
        worker.terminate()
        return

    print("✅ System Live. Latency should be minimal.")
    
    current_text = ""
    current_conf = 0.0
    warning = False
    
    try:
        while True:
            # 3. Get Frame (Non-blocking)
            frame = cam_thread.get_frame()
            if frame is None: 
                continue
            
            frame = cv2.flip(frame, 1)

            # 4. Send to AI (Fire and Forget)
            if frame_queue.empty():
                frame_queue.put(frame)

            # 5. Get AI Result (Non-blocking)
            if not result_queue.empty():
                current_text, current_conf, warning = result_queue.get()

            # 6. Draw UI
            h, w, _ = frame.shape
            cv2.rectangle(frame, (0, h-60), (w, h), (0,0,0), -1)
            
            if warning:
                 cv2.putText(frame, "Only 1 Hand!", (20, h-20), 
                             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            else:
                 cv2.putText(frame, current_text, (20, h-20), 
                             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            color = (0, 255, 0) if current_conf > 0.7 else (0, 165, 255)
            cv2.putText(frame, f"Conf: {current_conf:.2f}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            cv2.imshow("High Performance ASL", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        pass
    finally:
        cam_thread.stop()
        cv2.destroyAllWindows()
        worker.terminate()
        print("✅ Clean Exit")

if __name__ == "__main__":
    run_realtime_app()

🚀 Starting Fast Camera Thread...
📷 Trying connection to: WiFi IP...
✅ System Live. Latency should be minimal.
📷 Trying connection to: USB Index 1...
📷 Trying connection to: USB Index 0...
✅ SUCCESS: Connected to USB Index 0
✅ Clean Exit


In [2]:

for i in range(5):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera index {i} is available")
        cap.release()


Camera index 0 is available
Camera index 1 is available
